# This is a takehome challenge from Relax:

### The data is available as two attached CSV files:
takehome_user_engagement.csv
takehome_users.csv
### The data has the following two tables:
####  A user table ("takehome_users") with data on 12,000 users who signed up for the product in the last two years. This table includes:
● name: the user's name
● object_id: the user's id
● email: email address
● creation_source: how their account was created. This takes on one of 5 values:
○ PERSONAL_PROJECTS: invited to join another user's personal workspace
○ GUEST_INVITE: invited to an organization as a guest (limited permissions)
○ ORG_INVITE: invited to an organization (as a full member)
○ SIGNUP: signed up via the website
○ SIGNUP_GOOGLE_AUTH: signed up using Google Authentication (using a Google email account for their login
id)
● creation_time: when they created their account
● last_session_creation_time: unix timestamp of last login
● opted_in_to_mailing_list: whether they have opted into receiving
marketing emails
● enabled_for_marketing_drip: whether they are on the regular
marketing email drip
● org_id: the organization (group of users) they belong to
● invited_by_user_id: which user invited them to join (if applicable).

#### A usage summary table ("takehome_user_engagement") that has a row for each day that a user logged into the product.
Defining an "adopted user" as a user who has logged into the product on three separate days in at least one seven day period, identify which factors predict future user adoption.


In [285]:
import pandas as pd
import numpy as np
import datetime
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score,precision_recall_fscore_support,roc_auc_score


# load and explore data

In [69]:
activity = pd.read_csv('takehome_user_engagement.csv')
activity.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [5]:
activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [28]:
activity.visited.unique()

array([1])

In [13]:
!file takehome_users.csv

takehome_users.csv: ISO-8859 text, with CRLF line terminators


In [244]:
users = pd.read_csv('takehome_users.csv',encoding='ISO-8859-1',index_col=0)
users.head()

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
object_id,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [17]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 9 columns):
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(3), object(4)
memory usage: 937.5+ KB


# get label: adopted user (login three dif days during one 7 day period)

In [70]:
# label user using activity
# change timestamp to datetime object
activity['time_stamp'] = pd.to_datetime(activity['time_stamp'])

In [265]:
# total unique users
len(activity.user_id.unique())

8823

In [118]:
def is_adopted(data):
    """
    input data is sorted by time_stamp
    """
    if data.shape[0] < 3: # if total login times is less than 3 times, it's impossible for the user be adotped
            return 0
        
    for i in range(data.shape[0]-2): # start from one day

        ct = 1 # login times from a start day
        for j in range(i+1,data.shape[0]):# check following login time
            # if the next login is within 7 days
            if pd.Timedelta(data['time_stamp'].iloc[i] - data['time_stamp'].iloc[j]).days <= 7:
                ct += 1
            else:
                break
                
        if ct >= 3: # found one 7 day period that has 3 logins
            return 1
        
    return 0 # couldn't find any 7 day period that thas 3 logins

In [139]:
# group data into users
group_users = activity.groupby('user_id')

In [146]:
# assign label to users if it's adopted according to criteria
labels = group_users.apply(lambda x: is_adopted(x.sort_values(by='time_stamp',ascending=False)))
labels.head(5)

user_id
1    0
2    1
3    0
4    0
5    0
dtype: int64

# modeling with Xgboost

In [245]:
# Choose features that might affect user adoptibility
X = users[['creation_source',
       'last_session_creation_time', 'opted_in_to_mailing_list',
       'enabled_for_marketing_drip', 'org_id', 'invited_by_user_id']]

In [246]:
# convert catergorical data to number
X = pd.get_dummies(X)

In [247]:
X.head()

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
object_id,,,,,,,,,,
1,1.398139e+09,1,0,11,10803.0,1,0,0,0,0
2,1.396238e+09,0,0,1,316.0,0,1,0,0,0
3,1.363735e+09,0,0,94,1525.0,0,1,0,0,0
4,1.369210e+09,0,0,1,5151.0,1,0,0,0,0
5,1.358850e+09,0,0,193,5240.0,1,0,0,0,0


In [267]:
# assign labels to data, choose inner join to avoid 'na' values in y
Data = pd.concat([X,labels],axis=1,join='inner')

In [268]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8823 entries, 1 to 12000
Data columns (total 11 columns):
last_session_creation_time            8823 non-null float64
opted_in_to_mailing_list              8823 non-null int64
enabled_for_marketing_drip            8823 non-null int64
org_id                                8823 non-null int64
invited_by_user_id                    4776 non-null float64
creation_source_GUEST_INVITE          8823 non-null uint8
creation_source_ORG_INVITE            8823 non-null uint8
creation_source_PERSONAL_PROJECTS     8823 non-null uint8
creation_source_SIGNUP                8823 non-null uint8
creation_source_SIGNUP_GOOGLE_AUTH    8823 non-null uint8
0                                     8823 non-null int64
dtypes: float64(2), int64(4), uint8(5)
memory usage: 525.6 KB


In [269]:
Data.head(20)

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH,0
1,1.398139e+09,1,0,11,10803.0,1,0,0,0,0,0
2,1.396238e+09,0,0,1,316.0,0,1,0,0,0,1
3,1.363735e+09,0,0,94,1525.0,0,1,0,0,0,0
4,1.369210e+09,0,0,1,5151.0,1,0,0,0,0,0
5,1.358850e+09,0,0,193,5240.0,1,0,0,0,0,0
6,1.387424e+09,0,0,197,11241.0,1,0,0,0,0,0
7,1.356010e+09,0,1,37,NaN,0,0,0,1,0,0
10,1.401833e+09,1,1,318,4143.0,0,1,0,0,0,1
11,1.388117e+09,0,0,69,NaN,0,0,0,1,0,0
13,1.396196e+09,0,0,254,11204.0,0,1,0,0,0,0


In [270]:
# prepare X, y and training, testing data
X = Data.drop(0,axis=1)
y = Data[0]

In [271]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [272]:
# modeling with default setting
xgb = XGBClassifier()
xgbModel = xgb.fit(X_train,y_train)

In [273]:
# predict testing label
y_predict = xgbModel.predict(X_test)

In [278]:
# positive y values are about 19% in dataset, and default accuracy would be about 0.8
round(y.mean(),3),round(y_train.mean(),3),round(y_test.mean(),3)

(0.188, 0.19, 0.182)

In [311]:
def metrics(y_test,y_predict):
    print('accuracy:', round(accuracy_score(y_test,y_predict),4))
    print('confusion matrix:\n', confusion_matrix(y_test,y_predict))
    print('f1:', round(f1_score(y_test,y_predict),4))
    print('roc_auc:', round(roc_auc_score(y_test,y_predict), 4))
    
    report = precision_recall_fscore_support(y_test,y_predict,labels=[1])

    print('precision:', round(report[0][0],4))
    print('recall:', round(report[1][0],4))
    print('fscore:', round(report[2][0],4))
    print('support:', round(report[3][0],4))

In [312]:
# print model performance
metrics(y_test,y_predict)

accuracy: 0.8897
confusion matrix:
 [[2046  119]
 [ 173  309]]
f1: 0.6791
roc_auc: 0.7931
precision: 0.722
recall: 0.6411
fscore: 0.6791
support: 482


In [276]:
# print feature importance
"""
Insight: 
    Only the last_session_creation_time is the strongest factor that affect a user adoptibility.
    For the rest of features, creation_source with SIGNUP and GUEST_INVITE are mort important.
"""
sorted_idx = np.argsort(xgbModel.feature_importances_)[::-1]
for idx in sorted_idx:
    print(X_train.columns[idx], ':', xgbModel.feature_importances_[idx])

last_session_creation_time : 0.7007701
creation_source_SIGNUP : 0.07291784
creation_source_GUEST_INVITE : 0.068077415
invited_by_user_id : 0.045266572
org_id : 0.042954825
creation_source_PERSONAL_PROJECTS : 0.02480666
enabled_for_marketing_drip : 0.01624414
opted_in_to_mailing_list : 0.014519503
creation_source_ORG_INVITE : 0.014443017
creation_source_SIGNUP_GOOGLE_AUTH : 0.0
